In [8]:
!pip3 install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn.functional as F
import torch.nn as nn

from math import floor
import pandas as pd
import scipy.io as sio
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix,f1_score
import pytorch_lightning as pl

In [10]:
def load_hasc_ds(path, window, mode='train'):

    X, lbl = extract_windows_hasc(path, window, 5)

    if mode == "all":
        return X, lbl

    train_size = floor(0.8 * X.shape[0])
    print(train_size)
    if mode =="train":
        trainx = X[0:train_size]
        trainlbl =lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl>0)[0])))

        return testx, testlbl

def extract_windows_hasc(path, window_size, step):
    dataset = sio.loadmat(path+"hasc.mat")
    window_size
    windows = []
    lbl = []
    first = True
    num_cp = 0
    x = np.array(dataset['Y'])
    cp = np.array(dataset['L'])

    ts = np.sqrt(np.power(x[:, 0], 2) + np.power(x[:, 1], 2) + np.power(x[:, 2], 2))
    for i in range(0, ts.shape[0] - window_size, window_size // 5):
        windows.append(np.array(ts[i:i + window_size]))
        is_cp = np.where(cp[i:i + window_size] == 1)[0]
        if is_cp.size == 0:
            is_cp = [0]
        else:
            num_cp += 1
        lbl.append(is_cp[0])


    print("number of samples : {} /  number of samples with change point : {}".format(len(windows), num_cp))
    windows = np.array(windows)

    return windows, np.array(lbl)

def load_usc_ds(path, window, mode='train'):

    X, lbl = extract_windows_usc(path, window, mode)

    if mode == "all":
        return X, lbl
    train_size = int(floor(0.8 * X.shape[0]))
    if mode == "train":
        trainx = X[0:train_size]
        trainlbl = lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl > 0)[0])))

        return testx, testlbl


def extract_windows_usc(path, window_size, mode="train"):
    windows = []
    lbl = []
    dataset = sio.loadmat(path+"usc.mat")

    ts = np.array(dataset['Y'])
    ts = ts[:,0]
    cp = np.array(dataset['L'])
    cp = cp[:,0]

    num_cp = 0
    for i in range(0, ts.shape[0] - window_size, window_size // 5):
        windows.append(np.array(ts[i:i + window_size]))
        is_cp = np.where(cp[i:i + window_size] == 1)[0]
        if is_cp.size == 0:
            is_cp = [0]
        else:
            num_cp += 1
        lbl.append(is_cp[0])

    print("number of samples : {} /  number of samples with change point : {}".format(len(windows), num_cp))
    windows = np.array(windows)

    return windows, np.array(lbl)

def load_dataset(path, ds_name, win, bs, mode="train"):
    if ds_name == 'HASC':
        trainx, trainlbl = load_hasc_ds(path, window = 2 * win, mode=mode)
    elif ds_name == "USC":
        trainx, trainlbl = load_usc_ds(path, window=2 * win, mode=mode)
    else:
        raise ValueError("Undefined Dataset.")

    trainlbl = trainlbl.reshape((trainlbl.shape[0], 1))
    print(trainx.shape, trainlbl.shape)
    dataset = np.concatenate((trainlbl, trainx), 1)

    print("dataset shape : ", dataset.shape)
    if mode == "test":
        return dataset

    train_ds = TensorDataset(torch.from_numpy(dataset))
    return train_ds

In [11]:
def estimate_CPs(sim, gt, name, train_name, metric='cosine', threshold=0.5):
    est_cp = np.zeros(sim.shape[0])
    est_cp[np.where(sim < threshold)[0]] = 1
    tn, fp, fn, tp = confusion_matrix(gt, est_cp).ravel()
    f1 = f1_score(gt, est_cp)

    gt_id = np.where(gt == 1)[0]
    print("tn {}, fp {}, fn {}, tp {} ----- f1-score {}".format(tn, fp, fn, tp, f1))

    i = 1
    pos, seq_tp, seq_fn, seq_fp = 0, 0, 0, 0

    while i < gt.shape[0]:
        if gt[i] == 1:
            pos += 1
            j = i
            while gt[i] == 1:
                i += 1

            if np.sum(est_cp[j:i]) > 0:
                seq_tp += 1
                est_cp[j:i] = 0
            else:
                seq_fn += 1

        i += 1

    seq_fp = np.where(np.diff(est_cp) == 1)[0].shape[0]
    seq_f1 = (2 * seq_tp) / (2 * seq_tp + seq_fn + seq_fp)

    print("SEQ : Pos {}, fp {}, fn {}, tp {} ----- f1-score {}".format(pos, seq_fp, seq_fn, seq_tp, seq_f1))
    result = "tn, {}, fp, {}, fn, {}, tp, {}, f1-score, {}, Pos, {}, seqfp, {}, seqfn, {}, seqtp, {}, seqf1, {}\n".format(tn, fp, fn, tp, f1, pos, seq_fp, seq_fn, seq_tp, seq_f1)
    return result

In [12]:
class ResidualBlock(nn.Module):
    
    def __init__(self,
                 in_channels,
                 dilation_rate: int,
                 nb_filters: int,
                 kernel_size: int,
                 dropout_rate: float = 0, 
                 use_batch_norm: bool = False,
                 use_layer_norm: bool = False,
                 use_weight_norm: bool = False, 
                 training: bool = True):
        """Defines the residual block for the WaveNet TCN
        Args:
            dilation_rate: The dilation power of 2 we are using for this residual block
            nb_filters: The number of convolutional filters to use in this block
            kernel_size: The size of the convolutional kernel
            activation: The final activation used in o = Activation(x + F(x))
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        """

        self.dilation_rate = dilation_rate
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        # for causal padding
        self.padding = (self.kernel_size - 1) * self.dilation_rate
        
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        
        self.training = training

        super(ResidualBlock, self).__init__()
        
        self.conv_1 = nn.Conv1d(in_channels, self.nb_filters, self.kernel_size, 
                                padding=0, dilation=self.dilation_rate)        
        if self.use_weight_norm:
            weight_norm(self.conv_1) 
        self.bn_1 = nn.BatchNorm1d(self.nb_filters)
        self.ln_1 = nn.LayerNorm(self.nb_filters)              
        self.relu_1 = nn.ReLU()

        self.conv_2 = nn.Conv1d(self.nb_filters, self.nb_filters, self.kernel_size, 
                                padding=0, dilation=self.dilation_rate)        
        if self.use_weight_norm:
            weight_norm(self.conv_1)    
        self.bn_2 = nn.BatchNorm1d(self.nb_filters)
        self.ln_2 = nn.LayerNorm(self.nb_filters)              
        self.relu_2 = nn.ReLU()        
        
        self.conv_block = nn.Sequential()
        self.downsample = nn.Conv1d(in_channels, self.nb_filters, kernel_size=1) if in_channels != self.nb_filters else nn.Identity()
        
        self.relu = nn.ReLU()  
                
        self.init_weights()
        
        
    def init_weights(self):
        # in the realization, they use random normal initialization
        torch.nn.init.normal_(self.conv_1.weight, mean=0, std=0.05)
        torch.nn.init.zeros_(self.conv_1.bias)            
        
        torch.nn.init.normal_(self.conv_2.weight, mean=0, std=0.05)
        torch.nn.init.zeros_(self.conv_2.bias)            
        
        if isinstance(self.downsample, nn.Conv1d):         
            torch.nn.init.normal_(self.downsample.weight, mean=0, std=0.05)
            torch.nn.init.zeros_(self.downsample.bias)                    
            
    def forward(self, inp):
        # inp batch, channels, time
        ######################
        # do causal padding        
        out = F.pad(inp, (self.padding, 0))
        out = self.conv_1(out)
        
        if self.use_batch_norm:
            out = self.bn_1(out)
        elif self.use_layer_norm:
            out = self.ln_1(out)        
        out = self.relu_1(out)
        
        # spatial dropout
        out = out.permute(0, 2, 1)   # convert to [batch, time, channels]
        out = F.dropout2d(out, self.dropout_rate, training=self.training)        
        out = out.permute(0, 2, 1)   # back to [batch, channels, time]    
        
        #######################
        # do causal padding
        out = F.pad(out, (self.padding, 0))
        out = self.conv_2(out)
        if self.use_batch_norm:
            out = self.bn_2(out)
        elif self.use_layer_norm:
            out = self.ln_2(out)
        out = self.relu_2(out)            
        out = self.relu_2(out)    
        # spatial dropout
        # out batch, channels, time 
        
        out = out.permute(0, 2, 1)   # convert to [batch, time, channels]
        out = F.dropout2d(out, self.dropout_rate, training=self.training)
        out = out.permute(0, 2, 1)   # back to [batch, channels, time]            
        
        #######################        
        skip_out = self.downsample(inp)
        #######################
        res = self.relu(out + skip_out)
        return res, skip_out
    
# only causal padding
# only return sequence = True
    
class TCN(nn.Module):        
    def __init__(self,
                 in_channels=1,
                 nb_filters=64,
                 kernel_size=3,
                 nb_stacks=1,
                 dilations=(1, 2, 4, 8, 16, 32),
                 use_skip_connections=True,
                 dropout_rate=0.0, 
                 use_batch_norm: bool = False,
                 use_layer_norm: bool = False, 
                 use_weight_norm: bool = False):

        super(TCN, self).__init__()
        
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        self.in_channels = in_channels
        
        if self.use_batch_norm + self.use_layer_norm + self.use_weight_norm > 1:
            raise ValueError('Only one normalization can be specified at once.')        
        
        self.residual_blocks = []        
        res_block_filters = 0
        for s in range(self.nb_stacks):
            for i, d in enumerate(self.dilations):
                in_channels = self.in_channels if i + s == 0 else res_block_filters                
                res_block_filters = self.nb_filters[i] if isinstance(self.nb_filters, list) else self.nb_filters
                self.residual_blocks.append(ResidualBlock(in_channels=in_channels, 
                                                          dilation_rate=d,
                                                          nb_filters=res_block_filters,
                                                          kernel_size=self.kernel_size,
                                                          dropout_rate=self.dropout_rate, 
                                                          use_batch_norm=self.use_batch_norm,
                                                          use_layer_norm=self.use_layer_norm,
                                                          use_weight_norm=self.use_weight_norm))

        
        self.residual_blocks = nn.ModuleList(self.residual_blocks)
                                            
    def forward(self, inp):
        out = inp
        for layer in self.residual_blocks:
            out, skip_out = layer(out)
        if self.use_skip_connections:
            out = out + skip_out
        return out

########################### model #########################################
class Encoder(nn.Module):
    def __init__(self, c_in=1, nb_filters=64, kernel_size=4, 
                 dilations=[1,2,4,8], nb_stacks=2, n_steps=50, code_size=10, seq_len=100):       
        super(Encoder, self).__init__()        
        
        self.tcn_layer = TCN(in_channels=c_in, nb_filters=nb_filters, 
                             nb_stacks=nb_stacks, dilations=dilations, use_skip_connections=True, dropout_rate=0)
        
        self.fc1 = nn.Linear(nb_filters * seq_len, 2 * n_steps)  
        self.fc2 = nn.Linear(2 * n_steps, n_steps)    
        self.output_layer = nn.Linear(n_steps, code_size)           
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = x
        if len(out.shape) == 2:
            out = out.unsqueeze(1)
        out = self.tcn_layer(out)     
        out = out.flatten(1, 2)         
        out = self.relu(self.fc1(out)) 
        out = self.relu(self.fc2(out)) 
        out = self.output_layer(out)
        return out
    
########################### loss #########################################
def _cosine_simililarity_dim2(x, y):
    # x shape: (N, 1, C)
    # y shape: (1, 2N, C)
    # v shape: (N, 2N)
    cos = nn.CosineSimilarity(dim=2, eps=1e-6)
    v = cos(x.unsqueeze(1), y.unsqueeze(0))
    return v    

def nce_loss_fn(history, future, similarity, temperature=0.5):
    try:
        device = history.device
    except:
        device = 'cpu'
        
    criterion = torch.nn.BCEWithLogitsLoss()
    N = history.shape[0]
    sim = similarity(history, future)
    pos_sim = torch.exp(torch.diag(sim) / temperature)

    tri_mask = torch.ones((N, N), dtype=bool)
    tri_mask[np.diag_indices(N)] = False
    
    neg = sim[tri_mask].reshape(N, N - 1)    
    all_sim = torch.exp(sim / temperature)
    
    logits = torch.divide(torch.sum(pos_sim), torch.sum(all_sim, axis=1))
        
    lbl = torch.ones(history.shape[0]).to(device)

    loss = criterion(logits, lbl)    
    mean_sim = torch.mean(torch.diag(sim))
    mean_neg = torch.mean(neg)
    return loss, mean_sim, mean_neg


######################### preprocessing ###########################

def _history_future_separation(mbatch, win):    
    x = mbatch[:,1:win+1]
    y = mbatch[:,-win:]
    return x, y


################## PL wrapper ###############################################
class TSCP_model(pl.LightningModule):
    def __init__(
        self,
        model: nn.Module,     
        train_dataset: Dataset, 
        test_dataset: Dataset, 
        batch_size: int = 64,        
        num_workers: int = 2,        
        temperature: float = 0.1, 
        lr: float = 1e-4,
        decay_steps: int = 1000, 
        window_1: int = 100,
        window_2: int = 100
    ) -> None:
        super().__init__()
                    
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
        self.batch_size = batch_size
        self.num_workers = num_workers        
        
        self.temperature = temperature
        
        self.lr = lr
        self.decay_steps = decay_steps   
        
        self.window = window_1
        self.window_1 = window_1
        self.window_2 = window_2

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return self.model(inputs)
    
    def training_step(self, batch, batch_idx):

        history, future = _history_future_separation(batch[0], self.window)        
        history_emb = self.forward(history.float())
        future_emb = self.forward(future.float()) 

        history_emb = nn.functional.normalize(history_emb, p=2, dim=1)
        future_emb = nn.functional.normalize(future_emb, p=2, dim=1)

        train_loss, pos_sim, neg_sim = nce_loss_fn(history_emb, future_emb, similarity=_cosine_simililarity_dim2, 
                                                   temperature=self.temperature)

        return train_loss, pos_sim, neg_sim
        
    def validation_step(self, batch, batch_idx):

        history, future = _history_future_separation(batch, self.window)        
                
        history_emb = self.forward(history.float())
        future_emb = self.forward(future.float()) 
                
        history_emb = nn.functional.normalize(history_emb, p=2, dim=1)
        future_emb = nn.functional.normalize(future_emb, p=2, dim=1)
        

        val_loss, pos_sim, neg_sim = nce_loss_fn(history_emb, future_emb, similarity=_cosine_simililarity_dim2, 
                                                 temperature=self.temperature)      

        return val_loss, pos_sim, neg_sim


    def configure_optimizers(self) -> torch.optim.Optimizer:
        """Initialize optimizer.

        :return: optimizer for training CPD model
        """
        opt = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        return opt

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers
        )
    
def _cosine_simililarity_dim1(x, y):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    v = cos(x, y)
    return v    

In [13]:
DS_NAME = 'USC'
DATA_PATH = './data/'
OUTPUT_PATH = os.path.join('./output/', DS_NAME)
MODEL_PATH = os.path.join('./output/', "model")
LOSS = 'nce'
SIM = 'cosine'
GPU = 0

WIN = 100
CODE_SIZE = 10
BATCH_SIZE = 32
EPOCHS = 30
LR = 1e-4
TEMP = 0.5
TAU = 0.1
BETA = 1
EVALFREQ = 25
decay_steps = 1000


train_name = "CP2_model_" + DS_NAME + "_T" + str(TEMP) + "_WIN" + str(WIN) + \
             "_BS" + str(BATCH_SIZE) + "_CS" + str(CODE_SIZE) + "_lr" + str(LR) + \
             "_LOSS" + LOSS +  "_SIM" + SIM + "_TAU" + str(TAU) + "_BETA" + str(BETA)
print("------------------------------------>>> " + train_name)

train_ds = load_dataset(DATA_PATH, DS_NAME, WIN, BATCH_SIZE, mode = "train")
test_ds = load_dataset(DATA_PATH, DS_NAME, WIN, BATCH_SIZE, mode = "test")

------------------------------------>>> CP2_model_USC_T0.5_WIN100_BS32_CS10_lr0.0001_LOSSnce_SIMcosine_TAU0.1_BETA1
number of samples : 2336 /  number of samples with change point : 175
train samples :  1868
(1868, 200) (1868, 1)
dataset shape :  (1868, 201)
number of samples : 2336 /  number of samples with change point : 175
test shape (468, 200) and number of change points 35 
(468, 200) (468, 1)
dataset shape :  (468, 201)


In [14]:
prep_model = Encoder(code_size = CODE_SIZE, seq_len = WIN)


tscp_model = TSCP_model(prep_model, train_ds, test_ds, batch_size=BATCH_SIZE, temperature=TEMP, lr=LR, decay_steps=decay_steps, window_1=WIN)
optimizer = tscp_model.configure_optimizers()
lr = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=decay_steps)

In [15]:
train_loader = tscp_model.train_dataloader()
val_loader = tscp_model.val_dataloader()

train_losses = []
val_losses = []

epoch_wise_sim = []
epoch_wise_neg = []
for epoch in tqdm(range(EPOCHS)):
    
    iteration = 0
    train_losses_iters = []
    step_wise_sim = []
    step_wise_neg = []
    
    for index, batch in enumerate(train_loader):
            
        loss, sim, neg = tscp_model.training_step(batch, index)
        train_losses_iters.append(float(loss))
        step_wise_sim.append(float(sim))
        step_wise_neg.append(float(neg))
        if not iteration % 50:
            print("train_losses_iters", train_losses_iters[-1])

        loss.backward()
        optimizer.step()
        lr.step()
        optimizer.zero_grad()

        iteration += 1

        
        if not iteration % 50:
            tscp_model.eval()
            vall = []
            with torch.no_grad():
                for c, b in enumerate(val_loader):
                    val_loss, _, _ = tscp_model.validation_step(b, c)
                    vall.append(float(val_loss.detach()))
                    if c>10:
                        break
                print("val_loss", np.mean(vall))
                val_losses.append(np.mean(vall))
              
            
            tscp_model.train()
    print("epoch_train_loss", np.mean(train_losses_iters))  
    train_losses.append(np.mean(train_losses_iters))
    print("epoch_val_loss", np.mean(val_losses))
    epoch_wise_sim.append(np.mean(step_wise_sim))
    epoch_wise_neg.append(np.mean(step_wise_neg))

  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "


train_losses_iters 0.3132399618625641
val_loss 0.3040047499040763
train_losses_iters 0.12153904139995575


  3%|▎         | 1/30 [00:21<10:37, 21.97s/it]

epoch_train_loss 0.18944594630245434
epoch_val_loss 0.3040047499040763
train_losses_iters 0.11977645009756088
val_loss 0.2975579227010409
train_losses_iters 0.10160478204488754


  7%|▋         | 2/30 [00:44<10:26, 22.37s/it]

...

epoch_train_loss 0.03171652741730213
epoch_val_loss 0.2706637200455935
train_losses_iters 0.031927015632390976
val_loss 0.27151936416824657
train_losses_iters 0.0530737042427063


 97%|█████████▋| 29/30 [10:38<00:22, 22.21s/it]

epoch_train_loss 0.03558937305489839
epoch_val_loss 0.27069322501534016
train_losses_iters 0.03313778340816498
val_loss 0.2750374215344588
train_losses_iters 0.028403155505657196


100%|██████████| 30/30 [11:00<00:00, 22.01s/it]

epoch_train_loss 0.030513697130195166
epoch_val_loss 0.27083803156597747


In [17]:
torch.save({
            'model_state_dict': tscp_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, './tscp'+str(WIN)+str(CODE_SIZE)+str(BATCH_SIZE)+'_opt.pt')

In [18]:
x_test, lbl_test = test_ds[:,1:], test_ds[:,0]

num = x_test.shape[0]
lbl_test = np.array(lbl_test).reshape((lbl_test.shape[0], 1))
history = prep_model(torch.from_numpy(x_test[:, 0:WIN].reshape((num, 1, WIN))).float())
future = prep_model(torch.from_numpy(x_test[:, WIN:].reshape((num, 1, WIN))).float())
pred_out = np.concatenate((lbl_test, history.detach().numpy(), future.detach().numpy()), 1)
rep_sim = _cosine_simililarity_dim1(history, future)

print('Average similarity for test set : Reps : {}'.format(np.mean(rep_sim.detach().numpy())))
gt = np.zeros(lbl_test.shape[0])
gt[np.where((lbl_test > int(2 * WIN * 0.15)) & (lbl_test < int(2 * WIN * 0.85)))[0]] = 1

Average similarity for test set : Reps : 0.9715154767036438
tn 442, fp 0, fn 22, tp 4 ----- f1-score 0.2666666666666667
SEQ : Pos 7, fp 0, fn 5, tp 2 ----- f1-score 0.4444444444444444


In [43]:
result = estimate_CPs(rep_sim.detach().numpy(), gt, os.path.join(OUTPUT_PATH, train_name),
                    os.path.join(OUTPUT_PATH, "Evaluation.txt"),
                    metric='cosine', threshold=epoch_wise_sim[-1] - epoch_wise_neg[-1])

tn 436, fp 6, fn 8, tp 18 ----- f1-score 0.7199999999999999
SEQ : Pos 7, fp 4, fn 1, tp 6 ----- f1-score 0.7058823529411765
